In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
FB_EMAIL = os.getenv('FB_EMAIL')
FB_PASSWORD = os.getenv('FB_PASSWORD')

In [3]:
file_path = r'C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_groups_data.xlsx'
df1 = pd.read_excel(file_path, skiprows=1, header=0)

In [4]:
df1.head()

,SNO,PAGES,FACEBOOK ID,NO OF MEMBERS,REMARKS
0,1,Cyber Security Group - Pakistan,https://www.facebook.com/groups/596000380410413/,Public Â· 9K members Â· 5 posts a month,NaN
1,2,Pakistani moms in Canada ðŸ‡¨ðŸ‡¦,https://www.facebook.com/groups/1915352918584025/,Public Â· 9K members Â· 10+ posts a day,NaN
2,3,Electrical engineering world,https://www.facebook.com/groups/303168311067353/,Public Â· 96K members Â· 10+ posts a day,NaN
3,4,Gold Rate Pakistan Daily Update,https://www.facebook.com/groups/1236463527108778/,Public Â· 94K members Â· 3 posts a day,NaN
4,5,The Bankers Forum - (Pakistan),https://www.facebook.com/groups/TheBankersForum/,Public Â· 93K members Â· 6 posts a day,NaN


In [5]:
def wait_for_element_presence(driver, xpath, wait_secs=60, by=By.XPATH):
    try:
        print("wait_for_element_presence start")
        WebDriverWait(driver, wait_secs).until(
            EC.presence_of_element_located((by, xpath))
        )
        print("wait_for_element_presence end")
    except Exception as e:
        print(f"wait_for_element_presence Failed: {e} | XPath: {xpath}")

def login_to_facebook(driver):
    driver.get('https://www.facebook.com/login')
    
    # Wait for the login form to be present
    wait_after_navigate = '//div[@class="login_form_container"]'
    wait_for_element_presence(driver, wait_after_navigate)
    
    # Enter email and password
    driver.find_element(By.ID, "email").send_keys(FB_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FB_PASSWORD)
    driver.find_element(By.NAME, "login").click()
    
    # Wait for potential 2FA or the main page to be present
    wait_after_login = '//div[@role="banner"]//div[@aria-label="Facebook" and @role="navigation"]'
    wait_for_element_presence(driver, wait_after_login)
    
    print("Logged in to Facebook.")

def extract_member_count(driver):
    try:
        # Wait for the member count element to be present on the main page and extract its text
        member_count_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/members/') and contains(text(), 'members')]"))
        )
        member_count_text = member_count_element.text.split(' ')[0]
        return member_count_text
    except Exception as e:
        print(f"Error extracting member count: {e}")
        return None

def visit_group_and_extract_member_count(driver, group_url):
    try:
        driver.get(group_url)
        # Directly extract member count without any additional navigation or waiting
        member_count = extract_member_count(driver)
        return member_count
    except Exception as e:
        print(f"Error visiting group: {e}")
        return None

def main():
    # Set up the WebDriver and log in to Facebook once
    service = Service(r'C:\Users\hp\Downloads\chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    
    try:
        login_to_facebook(driver)
        
        group_urls = df1['FACEBOOK ID']
        results = []

        for url in group_urls:
            print(f"Visiting group: {url}")
            members_count = visit_group_and_extract_member_count(driver, url)
            results.append({'Group URL': url, 'Members Count': members_count})

        # Create a new DataFrame with the results
        result_df = pd.DataFrame(results)
        
        return result_df

    finally:
        driver.quit()

In [6]:
result_df = main()

wait_for_element_presence start
wait_for_element_presence end
wait_for_element_presence start
wait_for_element_presence end
Logged in to Facebook.
Visiting group: https://www.facebook.com/groups/596000380410413/
Visiting group: https://www.facebook.com/groups/1915352918584025/
Visiting group: https://www.facebook.com/groups/303168311067353/
Visiting group: https://www.facebook.com/groups/1236463527108778/
Visiting group: https://www.facebook.com/groups/TheBankersForum/
Visiting group: https://www.facebook.com/groups/185390198215162/
Visiting group: https://www.facebook.com/groups/pakindia.heritage/
Visiting group: https://www.facebook.com/groups/832137557346236/
Error extracting member count: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D3A6B632+29090]
	(No symbol) [0x00007FF6D39DE6E9]
	(No symbol) [0x00007FF6D389B1CA]
	(No symbol) [0x00007FF6D38EEFD7]
	(No symbol) [0x00007FF6D38EF22C]
	(No symbol) [0x00007FF6D39397F7]
	(No symbol) [0x00007FF6D391672F]
	(No symbol) [0x00007FF6D3

In [7]:
result_df.head()

,Group URL,Members Count
0,https://www.facebook.com/groups/596000380410413/,9.5K
1,https://www.facebook.com/groups/1915352918584025/,12.4K
2,https://www.facebook.com/groups/303168311067353/,134.1K
3,https://www.facebook.com/groups/1236463527108778/,95.7K
4,https://www.facebook.com/groups/TheBankersForum/,95.8K


In [8]:
result_df['Members Count'] = pd.to_numeric(result_df['Members Count'].str.replace('K', 'e3').str.replace('M', 'e6'))
result_df_sorted = result_df.sort_values(by='Members Count', ascending=True)
#result_df_sorted = result_df_sorted.reset_index(drop=True)

In [9]:
result_df_sorted['Group Name'] = df1['PAGES']

In [10]:
result_df_sorted.head(20)

,Group URL,Members Count,Group Name
52,https://www.facebook.com/groups/807147959841449/,81.0,PAKISTAN PROPERTY COUNCIL (0332-2638077)
53,https://www.facebook.com/groups/172955048884/,87.0,ISLAMIAN SOFTWARE ENGINEERING CLUB
46,https://www.facebook.com/groups/604520051520981/,154.0,Pakistan page ðŸ‡µðŸ‡°
58,https://www.facebook.com/groups/709133332510129/,815.0,Software Engineering Community of UOL
51,https://www.facebook.com/groups/245329859139746/,840.0,Civil Engineering
82,https://www.facebook.com/groups/pakistanirunne...,7600.0,Pakistani Runners Community
75,https://www.facebook.com/groups/Pakistansports...,8400.0,Pakistan Sports Sciences Council (PSSC)
68,https://www.facebook.com/groups/2977048692356255/,8500.0,Civil Engineer Problem Saloution
79,https://www.facebook.com/groups/pakistansteelu...,8500.0,Pakistan's Steel Update
83,https://www.facebook.com/groups/381794754332026/,8600.0,Pakistan petroleum Group


In [11]:
result_df_sorted.to_csv(r'C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_bot\new_csv.csv')

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
